# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load "cities.csv" into a dataframe
df = pd.read_csv("output_data/cities.csv")
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.6000,73.0833,83.34,67,100,7.90,MV,1633797558
1,Busselton,-33.6500,115.3333,46.09,80,16,8.14,AU,1633797559
2,Saint Paul Harbor,57.7900,-152.4072,49.51,46,90,9.22,US,1633797560
3,Sitka,57.0531,-135.3300,48.09,76,90,17.27,US,1633797562
4,New Norfolk,-42.7826,147.0587,47.70,62,44,3.00,AU,1633797563


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Store humidity data # and convert to float
humidity = df["Humidity"]#.astype(float)

In [4]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,41.09,64,93,4.97,AU,1633291202
1,Barrow,71.2906,-156.7887,30.22,80,90,12.66,US,1633291187
2,Nizhniy Kuranakh,58.8353,125.4869,19.71,97,38,3.33,RU,1633291208
3,Albany,42.6001,-73.9662,66.09,88,100,1.30,US,1633290937
4,Teberda,43.4456,41.7448,48.06,97,100,4.47,RU,1633291212


In [7]:
# Narrow down the cities to fit the following conditions:
# max temperature in the range of 70 - 80
# Wind speed < 10 mph
# Cloudiness = 0
cities_df = df.loc[(df['Max Temp']>70) & (df['Max Temp']<80) & (df['Wind Speed']<10) & (df['Cloudiness'] ==0), :]
hotel_df = cities_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Lat,Lng
24,Kissamos,GR,35.4939,23.6569
215,Valle de Allende,MX,26.9333,-105.4000
323,Quimper,FR,48.0000,-4.1000
342,Lichinga,MZ,-13.3128,35.2406
350,Rāwah,IQ,34.4816,41.9093
431,Tura,IN,25.5198,90.2201
494,Alexandria,EG,31.2156,29.9553
597,Beloha,MG,-25.1667,45.0500


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# set up a parameters dictionary that will be updated with new city each iteration
target_radius = 5000
target_type = "hotel"
params = {
#     "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through the hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']

    # update address key value
    params['location'] = f"{lat},{lng}"
    try:
        # make request
        hotels = requests.get(base_url, params=params)
    
        # convert to json
        hotels = hotels.json()
        
        # store hotel name back to the hotel_df dataframe
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][1]['name']
        
        # Replace the lat and lng in the dataframe to the hotels location
        hotel_df.loc[index, "Lat"] = hotels['results'][1]['geometry']['location']['lat']
        hotel_df.loc[index, "Lng"] = hotels['results'][1]['geometry']['location']['lng']
        print(f"{row['City']} has more than 1 hotels.")
        
    except:
        print(f"{row['City']} has no hotels in 5000m range.")
# Visualize to confirm lat lng appear
hotel_df

Kissamos has more than 1 hotels.
Valle de Allende has more than 1 hotels.
Quimper has more than 1 hotels.
Lichinga has more than 1 hotels.
Rāwah has more than 1 hotels.
Tura has more than 1 hotels.
Alexandria has more than 1 hotels.
Beloha has more than 1 hotels.


,City,Country,Lat,Lng,Hotel Name
24,Kissamos,GR,35.497683,23.661935,Galini Beach Hotel
215,Valle de Allende,MX,26.933597,-105.399887,University Cultural Center of Valle de Allende
323,Quimper,FR,47.994284,-4.099868,Hôtel Escale Oceania Quimper
342,Lichinga,MZ,-13.316785,35.243501,ATM Barclays Bank
350,Rāwah,IQ,34.469569,41.918180,Rawa General Hospital
431,Tura,IN,25.513303,90.217293,"Click N Browse Internet Cafe, Sukbhir Bhawan, ..."
494,Alexandria,EG,31.216067,29.938677,Royal Crown Hotel
597,Beloha,MG,-25.173539,45.055916,Ronald Prod Gasy


In [9]:
# remove the city that doesn't have a hotel in 5000m range
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
24,Kissamos,GR,35.497683,23.661935,Galini Beach Hotel
215,Valle de Allende,MX,26.933597,-105.399887,University Cultural Center of Valle de Allende
323,Quimper,FR,47.994284,-4.099868,Hôtel Escale Oceania Quimper
342,Lichinga,MZ,-13.316785,35.243501,ATM Barclays Bank
350,Rāwah,IQ,34.469569,41.918180,Rawa General Hospital
431,Tura,IN,25.513303,90.217293,"Click N Browse Internet Cafe, Sukbhir Bhawan, ..."
494,Alexandria,EG,31.216067,29.938677,Royal Crown Hotel
597,Beloha,MG,-25.173539,45.055916,Ronald Prod Gasy


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))